In [1]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST

from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive,EWC
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, Resize


import wandb
import timm

import avalanche

/home/grbagwe/miniconda3/envs/fedconlearn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [15]:

train_transform = Compose([
#     Resize(224),
    ToTensor(),
])

test_transform = Compose([
#     Resize(224),
    ToTensor(),
])


benchmark = avalanche.benchmarks.SplitCIFAR10(n_experiences=5, return_task_id=False, 
                       train_transform=train_transform,eval_transform = test_transform
                      )

Files already downloaded and verified
Files already downloaded and verified


In [22]:
benchmark.get_reproducibility_data()

{'class_ids_from_zero_from_first_exp': False,
 'class_ids_from_zero_in_each_exp': False,
 'class_mapping': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'classes_order': [7, 3, 9, 0, 5, 8, 6, 2, 1, 4],
 'classes_order_original_ids': [7, 3, 9, 0, 5, 8, 6, 2, 1, 4],
 'n_classes_per_exp': [2, 2, 2, 2, 2],
 'n_experiences': 5,
 'has_task_labels': False}

In [23]:
benchmark.original_classes_in_exp

[{3, 7}, {0, 9}, {5, 8}, {2, 6}, {1, 4}]

In [7]:
# model = timm.models.vit_tiny_patch16_224(pretrained=True, num_classes=benchmark.n_classes)
model = SimpleMLP(num_classes=benchmark.n_classes)


In [12]:
model = avalanche.models.SimpleCNN(num_classes= benchmark.n_classes)

In [13]:
model

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.25, inplace=False)
    (12): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (13): ReLU(inplace=True)
    (14): AdaptiveMaxPool2d(output_size=1)
    (15): Dropout(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [14]:
# loggers 
loggers = []

# log to Tensorboard
loggers.append(TensorboardLogger())

# log to text file
loggers.append(TextLogger(open('log.txt', 'a')))

# print to stdout
loggers.append(InteractiveLogger())

# W&B logger - comment this if you don't have a W&B account
loggers.append(WandBLogger(project_name="avalanche", run_name="ewc-SplitCifar-MLP"))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=True,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=loggers
)


In [16]:
# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy = EWC(
    model,
    optimizer=SGD(model.parameters(), lr=0.001, momentum=0.9),
    criterion=CrossEntropyLoss(),
    ewc_lambda=0.4,
    train_mb_size=50, train_epochs=5,
    eval_mb_size=50,device='cuda',
    evaluator=eval_plugin)

In [17]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start training on experience ", experience.current_experience)

    cl_strategy.train(experience)
    print("End training on experience", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(cl_strategy.eval(benchmark.test_stream[:]))
        
        
# for experience in benchmark.train_stream:
#     # train returns a dictionary which contains all the metric values
#     res = cl_strategy.train(experience)
#     print('Training completed')

#     print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))
    


Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 50.81it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 8504.2383
	DiskUsage_MB/train_phase/train_stream/Task000 = 8503.4814
	Loss_Epoch/train_phase/train_stream/Task000 = 1.4679
	Loss_MB/train_phase/train_stream/Task000 = 0.7162
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0013
	Time_Epoch/train_phase/train_stream/Task000 = 3.9313
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4263
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4800
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 54.00it/s]
Epoch 1 ended.
	

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 60.22it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002 = 1496.1997
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp002 = 11894.0547
	Loss_Exp/eval_phase/test_stream/Task000/Exp002 = 6.8166
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 58.64it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 1467.3171
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003 = 11935.2666
	Loss_Exp/eval_phase/tes

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 60.42it/s]
> Eval on experience 4 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 1461.2166
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 15871.2637
	Loss_Exp/eval_phase/test_stream/Task000/Exp004 = 7.5256
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004 = 0.0000
-- >> End of eval phase << --
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x7fd916181520>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 15871.2637
	Loss_Stream/eval_phase/test_stream/Task000 = 6.3197
	StreamForgetting/eval_phase/test_stream = 0.5000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1000
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|███

	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 1460.3458
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 19975.6592
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.5000
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 6.3559
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 53.97it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 1459.5586
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 20006.5312
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.5000
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 5.1859
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Ta

	DiskUsage_MB/train_phase/train_stream/Task000 = 22704.5537
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7319
	Loss_MB/train_phase/train_stream/Task000 = 0.7207
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0002
	Time_Epoch/train_phase/train_stream/Task000 = 6.1370
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4994
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.5800
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:06<00:00, 32.26it/s]
Epoch 3 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 23497.2979
	DiskUsage_MB/train_phase/train_stream/Task000 = 23497.2979
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7235
	Loss_MB/train_phase/train_stream/Task000 = 0.7219
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 6.1881
	Top1_Acc_Epoch/train_phase/train_str

	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 1489.1429
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004 = 24708.7988
	Loss_Exp/eval_phase/test_stream/Task000/Exp004 = 7.7633
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004 = 0.0000
-- >> End of eval phase << --
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x7fd900538070>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 24708.7988
	Loss_Stream/eval_phase/test_stream/Task000 = 5.6843
	StreamForgetting/eval_phase/test_stream = 0.5000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1000
Start training on experience  4
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:07<00:00, 27.91it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 25547.4707
	DiskUsage_M

	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 29085.1211
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.5000
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 7.6690
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 59.11it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 1481.3539
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 29121.1260
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.5000
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 7.8200
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|█████████████████████████████████

In [18]:
import wandb
wandb.finish()

CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000,▂▃▁█▅
CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001,▁▇▅▅█
CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002,█▂▃▇▁
CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003,██▁▅▅
CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004,██▁█▇
DiskUsage_Epoch/train_phase/train_stream/Task000,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000,▁▃▄▆█
DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001,▁▃▄▆█
DiskUsage_Exp/eval_phase/test_stream/Task000/Exp002,▁▃▄▆█
DiskUsage_Exp/eval_phase/test_stream/Task000/Exp003,▁▃▄▆█
DiskUsage_Exp/eval_phase/test_stream/Task000/Exp004,▁▃▄▆█
